In [1]:
from mlagents.torch_utils.torch import torch
import numpy as np

from mlagents.plugins.bvh_utils.visualize import skeletons_plot, motion_animation
from mlagents.plugins.dataset.dataset import SkeletonInfo
import mlagents.plugins.bvh_utils.lafan_utils as utils
from mlagents.plugins.dataset.skeleton_side_channel import Skeleton_SideChannel 

import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib qt


from mlagents_envs.environment import UnityEnvironment

# INITIALIZATION OF UNITY SAVED DATA AND UNITY ENV

In [2]:
motion_path = "../ml-agents/mlagents/plugins/skeleton_aware_op/data/LaFan/Real_Data/"
# motion_path = "../../LaFan_BVH/rotation_test/"
skdata = SkeletonInfo(motion_path, xyz='xyz')

In [18]:
# Setup Unity Environment + sidechannel
skeleton_sidechannel = Skeleton_SideChannel()

try:
    env.close()
except:
    pass

# filename = None enables to communicate directly with the unity editor
env = UnityEnvironment(file_name=None, seed=1, side_channels=[skeleton_sidechannel])
env.reset()

[]


# DATA RETRIVAL FROM FILE AND ENV

In [6]:
with open('walk1_subject1_csv.txt') as f:
    lines = f.readlines()

n_lines = len(lines)
n_features = len(lines[0].split(','))

features = torch.zeros((n_lines,n_features)).float()

for i, line in enumerate(lines):
    line = line
    line_array = [float(f) for f in line.split(',')]
    features[i] = torch.tensor(line_array).float()


In [9]:
skdata.parents

array([-1,  0,  1,  2,  3,  0,  5,  6,  7,  0,  9, 10, 11, 12, 11, 14, 15,
       16, 11, 18, 19, 20])

In [19]:
behavior_name = list(env.behavior_specs)[0]
spec = env.behavior_specs[behavior_name]
env.reset()

# get action and obs dimension
state_dim = spec.observation_specs[0].shape[0]
action_dim = spec.action_spec.continuous_size

decision_steps, terminal_steps = env.get_steps(behavior_name)

unity_data = []

for i in range(60):
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    unity_data.append(decision_steps.obs[0])
    
    # Generate an action for all agents
    action = spec.action_spec.empty_action(len(decision_steps))

    # Set the actions
    env.set_actions(behavior_name, action)

    # Move the simulation forward
    env.step()

# DATA PROCESSING

In [7]:
features  = features.reshape(features.shape[0],skdata.num_joints,7)

csv_positions = features[:,:,:3]
csv_rotations = features[:,:,3:]
csv_local_positions = csv_positions[:] - csv_positions[:,0:1,:]

In [21]:
n = 54

features_per_joint = 13

obs = torch.cat([torch.tensor(unity_data[f]).float() for f in range(60)],dim=0).float()
print(obs.shape)

# extract only the observation corresponding to the joints
joint_features = obs[0:n,:-6]
print(joint_features.shape)
joint_features = joint_features.reshape(n,-1, 13)

init_rotations = torch.tensor(skeleton_sidechannel.msg[:22*4]).float().reshape(22,4)
init_positions = torch.tensor(skeleton_sidechannel.msg[22*4:]).float().reshape(22,3)

# extract information from the observation vector
env_velocity = joint_features[:,:,:3].clone()
env_angular_vel = joint_features[:,:,3:6].clone()
env_positions = joint_features[:,:,6:9].clone()
env_rotations = joint_features[:,:,9:].clone()

# unity rotation is x,y,z,w and lafan is w,x,y,z, change to the latter
temp = env_rotations[:,:,3].clone()
env_rotations[:,:,1:] = env_rotations[:,:,:3] 
env_rotations[:,:,0] = temp

env_local_positions = env_positions[:] - env_positions[:,0:1,:]

torch.Size([59, 292])
torch.Size([54, 286])


In [22]:
# reconstruct pose from rotations
offsets = init_positions.clone()
offsets = offsets.reshape(1,22,3)
offsets = offsets.repeat(env_rotations.shape[0],1,1)

env_rotations[:,0,:] = torch.tensor([1.,0.,0.,0.]).float()
csv_rotations[:,0,:] = torch.tensor([1.,0.,0.,0.]).float()

_, env_pos_from_rot = utils.quat_fk(env_rotations, offsets, skdata.parents)
_, csv_pos_from_rot = utils.quat_fk(csv_rotations[:env_rotations.shape[0],:,:], offsets, skdata.parents)


# VISUALIZE

In [28]:
# POSE FROM POSITIONS
limits = [[-1,1],[-1,1],[-1,1]]
# fig, ax = skeletons_plot([csv_local_positions[100].cpu().detach(), env_local_positions[20].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
fig,ax=skeletons_plot([csv_local_positions[0].cpu().detach()], [skdata.edges], ['g'], limits=limits, return_plot=True)

plt.show()

In [24]:
# POSE FROM ROTATIONS
limits = [[-1,1],[-1,1],[-1,1]]
fig,ax=skeletons_plot([csv_pos_from_rot[0].cpu().detach(), env_pos_from_rot[1].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
plt.show()

In [38]:
# MOTION FROM POSITIONS
anim = motion_animation([csv_local_positions[:55].cpu().detach(), env_local_positions.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits)

c:\users\nicol\work\master\dissertation\ml-agents\ml-agents\mlagents\plugins\bvh_utils\visualize.py:151: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax = p3.Axes3D(fig)


In [25]:
env.close()